<a href="https://colab.research.google.com/github/Aakash326/ML-models/blob/main/poem_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
corpus = [
    "the sun is shining bright",
    "the sky is blue and clear",
    "birds are singing in the trees",
    "flowers bloom in the morning light",
    "the world wakes with gentle grace"
]

In [10]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words=len(tokenizer.word_index)+1

In [11]:
len(tokenizer.word_index)+1

24

In [7]:
tokenizer.word_index

{'the': 1,
 'is': 2,
 'in': 3,
 'sun': 4,
 'shining': 5,
 'bright': 6,
 'sky': 7,
 'blue': 8,
 'and': 9,
 'clear': 10,
 'birds': 11,
 'are': 12,
 'singing': 13,
 'trees': 14,
 'flowers': 15,
 'bloom': 16,
 'morning': 17,
 'light': 18,
 'world': 19,
 'wakes': 20,
 'with': 21,
 'gentle': 22,
 'grace': 23}

In [16]:
seq=[]
for line in corpus:
  token=nltk.word_tokenize(line.lower())
  token_id=tokenizer.texts_to_sequences([" ".join(token)])[0]
  for i in range(1,len(token_id)):
    n_gram_seq = token_id[:i + 1]
    seq.append(n_gram_seq)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [31]:
import nltk
nltk.download('punkt')  # ✅ Download tokenizer model

from tensorflow.keras.preprocessing.text import Tokenizer

corpus = [
    "the sun is shining bright",
    "the sky is blue and clear",
    "birds are singing in the trees",
    "flowers bloom in the morning light",
    "the world wakes with gentle grace"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

input_sequences = []
for line in corpus:
    # tokenized_line = nltk.word_tokenize(line)
    token_ids = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_ids)):
        n_gram_seq = token_ids[:i + 1]
        input_sequences.append(n_gram_seq)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
max_seq_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

In [37]:
x=input_sequences[:,:-1]
y=input_sequences[:,-1]
y=to_categorical(y,num_classes=total_words)

In [34]:
model=Sequential()
model.add(Embedding(total_words, 10, input_length=max_seq_len - 1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=200, verbose=1)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: 3.1788
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.0000e+00 - loss: 3.1771
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.0000e+00 - loss: 3.1753
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.0833 - loss: 3.1735
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.0833 - loss: 3.1716
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0833 - loss: 3.1697
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.0833 - loss: 3.1677
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.0833 - loss: 3.1655
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.0833 - loss: 3.1632
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.0833 - loss: 3.1606
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0833 - loss: 3.1579
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy

In [56]:
def pred_next_words(seed,maxlen=max_seq_len):
  token_list = tokenizer.texts_to_sequences([seed.lower()])[0]
  token_list=pad_sequences([token_list],maxlen=maxlen-1,padding='pre')
  predict=model.predict(token_list,verbose=0)
  predict_idx=np.argmax(predict)
  predict_word=tokenizer.index_word[predict_idx]
  return predict_word

In [59]:
def generate_line(seed_text, max_words=10, max_len=max_seq_len):
    for _ in range(max_words):
        token_list = tokenizer.texts_to_sequences([seed_text.lower()])[0]
        token_list = pad_sequences([token_list], maxlen=max_len - 1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_word = tokenizer.index_word.get(predicted_index)

        if predicted_word is None:
            break  # Stop if word is not found (rare but possible)

        seed_text += ' ' + predicted_word

        if predicted_word in ['.', '?', '!']:
            break  # Optional: Stop if a sentence-ending word is predicted
    return seed_text


In [63]:
seed='the sky is'
pred_next_words(seed)

'blue'

In [62]:
seed='the sky is'
generate_line(seed)

'the sky is blue and clear clear clear clear light and and and'